# Final Project E-Commerce: Product Range Analysis
## Household online store

#### Project Goal:
To analyze the current product range of the household online store, identify opportunities for optimization, and develop a set of actionable recommendations to increase sales and profitability. By optimizing the product range, we aim to improve the customer experience, increase customer satisfaction, and ultimately drive business growth.

### Description of the data:

The dataset contains the transaction history of an online store that sells household goods.

The file `ecommerce_dataset_us.csv` contains the following columns:

`InvoiceNo` — order identifier

`StockCode` — item identifier

`Description` — item name

`Quantity`

`InvoiceDate` — order date

`UnitPrice` — price per item

`CustomerID`

## 1. Define the research question and objectives:


We ask some question in order to understand the current business situation:

- What is the current product range of the E-Commerce business?  
- Which products are the most popular and profitable?  
- Which products are not performing well and should be discontinued or improved?  
- How can the product range be optimized to increase sales and profitability?  
- Can we identify statistically significant differences between the performance of different product categories or price ranges?
- Can we use a machine learning model to predict product demand and optimize the product range?

Let's dive deep into the data in order to find some intesting findings:

## 2. Check the data:  

### Import libraries:

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from plotly import graph_objects as go
from scipy import stats as st
from scipy.stats import pearsonr
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import math as mth
from statsmodels.stats.proportion import proportions_ztest
import warnings

warnings.filterwarnings("ignore") 

### Import data:

In [ ]:
path='/Users/barbrender/Library/CloudStorage/GoogleDrive-barbrender@gmail.com/My Drive/UDEMY/practicum/The final project'
try:
    data= pd.read_csv(path + '/ecommerce_dataset_us.csv',sep='\t')
except:
    data= pd.read_csv('/datasets/ecommerce_dataset_us.csv',sep='\t')

### Review the data to ensure that they are complete and accurate.

In [ ]:
data.info()

#### Changing the data type and columns names:

In [ ]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data['CustomerID'] = data['CustomerID'].astype('object')

data.columns = data.columns.str.lower()

data.rename(columns={'invoiceno': 'invoice_no',
                     'stockcode': 'stock_code',
                     'invoicedate': 'invoice_date',
                     'unitprice' : 'unit_price',
                     'customerid': 'customer_id'}, inplace=True)

In [ ]:
## adding date column
data['date'] = data['invoice_date'].dt.date
data['date'] = pd.to_datetime(data['date'])
data['month'] = data['invoice_date'].dt.strftime('%Y-%m')
data['day_of_week'] = data['date'].dt.dayofweek  # Monday is 0, Sunday is 6
data['weekday_or_weekend'] = data['day_of_week'].apply(lambda x: 'Weekday' if x < 5 else 'Weekend')
data['revenue'] = data['unit_price'] * data['quantity']

In [ ]:
data.info()

In [ ]:
data.head(10)

In [ ]:
data.describe(include='all')

**Initial findings:**
- Our data starts on 2018-11-29 and ends on 2019-12-07 - total of a year and a week.
- We have 2 columns with missing values - description, customer_id.
- We have 25,900 unique invoices.
- 4070 unique products.
- 4372 unique customers.
- Our top user is 17841 with 169 orders (has 7983 rows in our data).
- Invoice 573585 is top invoice with 1114 rows (It has no customer id related) - we should check this.
- WHITE HANGING HEART T-LIGHT HOLDER (85123A) is the best seller product (with 2369 rows).
- The stock_code appears less than the description so we need to check why?!
- The best sale date is '2019-10-29' (it's the day invoice 573585 was made) - we should check this.
- Weekday is more common then weekends.

** **

**Statistical findings:**
- Avg quantity is 9.55.
- Avg price is 4.6.
- The median is lower than the avg and the min/max price and quantity has unusual values - we should check for outliers.

### Identify any missing data, outliers or inconsistencies in the data.

In [ ]:
data.invoice_date.max()

In [ ]:
data.invoice_date.min()

The data contains info on more than one year.  
Let's investigate one year.

In [ ]:
data = data.query('"2018-12-01" < invoice_date < "2019-12-01"')

#### Checking quantity and unit price columns:
#### Quantity column:

In [ ]:
data[['unit_price','quantity']].describe()

There are prices and quantities that might be outliers.  
We should investigate more.

Checking negative values:

In [ ]:
data.query('(quantity < 0) & (unit_price < 0)')

In [ ]:
data.query('quantity < 0').sample(10)

All the invoices starts with 'C' are returned items as for correction.  
Let's check the other invoices:

In [ ]:
data.query('(quantity < 0) & (~invoice_no.str.startswith("C"))')['description'].unique()

In [ ]:
len(data.query('quantity < 0'))

In [ ]:
print('Total revenue for negative quantity:','{:.2f}'.format(data.query('quantity < 0')['revenue'].sum()))

In [ ]:
print('Products with negative quantity:',data.query('quantity < 0')['description'].nunique())

In [ ]:
print('Products with negative quantity - not returned:')
data.query('(quantity < 0) & (~invoice_no.str.startswith("C"))')['description'].nunique()

There are 10298 transactions that had corrections (returned or other issues) - they have negative revenue.  
The revenue is: 691,185.8.  
2070 products are damaged, lost, wrong or something similar.  
132 of them are returned products.  
We can add a filter so we can check the revenue influence.

In [ ]:
data['damaged'] = np.where((data.quantity < 0) & (~data.invoice_no.str.startswith("C")), 1, 0)

In [ ]:
data['return'] = np.where(data.invoice_no.str.startswith("C"), 1, 0)

In [ ]:
data.query('(quantity < 0)').head(10)

In [ ]:
data['damaged'].value_counts(normalize=True)

In [ ]:
data[data['damaged'] == 1]['revenue'].sum()

The damaged products are only 2% and 0 revenue.  
We can drop them.

In [ ]:
data = data[data['damaged'] == 0]

In [ ]:
data = data.drop('damaged', axis=1)
data.head()

In [ ]:
data['return'].value_counts(normalize=True)

In [ ]:
print('Total revenue from returned product:','{:.2f}'.format(data[data['return'] == 1]['revenue'].sum()))

We will continue to investigate the returned items later.

Checking max and min quantity:

**MAX quantity:**

In [ ]:
data.sort_values('quantity', ascending=False).head(10)

**MIN quantity:**

In [ ]:
data.sort_values('quantity').head(10)

We can remove the two rows that are the same and sums to 0, it sesms like the correct each other.  
They can effect our revenue. 

In [ ]:
data.drop([61624,61619], axis=0, inplace=True)

#### Unit_price column:

**MAX price:**

In [ ]:
data.sort_values('unit_price', ascending=False).head(10)

We can see some special prices we should investigate.

**MIN price:**

In [ ]:
data.sort_values('unit_price').head(10)

In [ ]:
data[data.unit_price <0]

We should remove the row: 'Adjust bad debt' as they can influence on the revenue.  
I will investigate it more and them remove it.

Let's look at the 0 price items:

In [ ]:
data[data.unit_price == 0].sample(10)

In [ ]:
len(data[data.unit_price == 0])

There are 1151 rows with 0 price, they might be due to special sales like buy 2 get 1 free ect.  
We should investigate it the lower case products, but first we will check the misssing values:

#### Checking for missing values:

In [ ]:
data.isna().sum()

In [ ]:
data.isna().sum()/len(data)

**Description column:**

In [ ]:
data[data.description.isna()]

In [ ]:
data[data.description.isna()].query('unit_price != 0')

In [ ]:
data[data.description.isna()]['stock_code'].value_counts()

All the missing descriptions are 0 priced, they don't have influence on the revenue.  
I will fill the missing values with common description according to the stock_code.

In [ ]:
data['description'] = data.groupby('stock_code')['description'].transform(lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0])

In [ ]:
data.description.isna().sum()

All other missing values I will fill with 'unknown':

In [ ]:
data['description'] = data['description'].fillna('unknown')

In [ ]:
data.description.isna().sum()

**Customer_id column:**

In [ ]:
data[data.customer_id.isna()].sample(10)

There are many invoices with no customer id, we can assume these are non registered customers there is no pattern.  
I will assign them the value - unregister.

In [ ]:
data['customer_id'] = data['customer_id'].fillna('unregister')

In [ ]:
data.customer_id.isna().sum()

After dealing with the missing values, let's check lower case values in description:

In [ ]:
lower_case_items = data[data.description.str.islower()]
lower_case_items

In [ ]:
lower_case_items['description'].unique()

All these rows are unknown products and priced 0 we can remove them.  
We can remove all the zero priced items as they won't influence our revenue.

In [ ]:
data = data[data.unit_price !=0]
data.head()

#### Checking the product WHITE HANGING HEART T-LIGHT HOLDER product:

We found out the the is inconsistency between the stock_code and the product name and that there are stock_codes with lower and upper case:

In [ ]:
data.query('description == "WHITE HANGING HEART T-LIGHT HOLDER"')['stock_code'].value_counts()

In [ ]:
data[data['stock_code'].str.islower()].sample(10)

We found inconsistencies in the product code.  
There are some products codes with lower case and some with upper case.  
We can't find any pattern.  
We will change all stock_code to upper case.

In [ ]:
data['stock_code'] = data.stock_code.str.upper()

In [ ]:
data.query('description == "WHITE HANGING HEART T-LIGHT HOLDER"')['stock_code'].value_counts()

In [ ]:
data[['stock_code','description']].describe(include='all')

We can see difference in the unique codes copmared to the description.  
Let's check it:

In [ ]:
product_name_counts = data.groupby('stock_code')['description'].nunique()
product_name_counts[product_name_counts > 1]

In [ ]:
data[data.stock_code== '85184C']['description'].value_counts()

In [ ]:
data.loc[(data['stock_code'] == '85184C') & (data['description'] == 'SET 4 VALENTINE DECOUPAGE HEART BOX'), 'description'] = 'S/4 VALENTINE DECOUPAGE HEART BOX'

In [ ]:
product_name_counts = data.groupby('stock_code')['description'].nunique()
product_name_counts[product_name_counts > 1]

Let's check the description too:

In [ ]:
product_code_counts = data.groupby('description')['stock_code'].nunique()
product_code_counts[product_code_counts > 1]

There are some products with more than one stock_codes.  
Theren't many of them and we can't tell who's code is the right one, I will leave it for now.  

**Conclusion:**  
We changed all the stock_codes to upper case.  
We managed to fix the freq difference between the stock_code and the description.  
We still have some products with same name and different stock_code.

#### Checking duplicates:

In [ ]:
data.duplicated().sum()

In [ ]:
data[data.duplicated()].sort_values('invoice_no').sample(10)

We found 4961 duplicated rows, this rows might be a product that added twice but it doesn't mean it's wrong.  
We choose to leave these rows as they are.  
We just need to remember to sum the quantity later while calculating the revenue.

**Looking for non numeric stock_codes:**

In [ ]:
sorted(data.stock_code.unique(), reverse=True)[0:25]

In [ ]:
non_sale = ['S',
 'POST',
 'PADS',
 'M',
 'GIFT_0001_50',
 'GIFT_0001_40',
 'GIFT_0001_30',
 'GIFT_0001_20',
 'GIFT_0001_10',
 'DOT',
 'DCGSSGIRL',
 'DCGSSBOY',
 'DCGS0076',
 'DCGS0070',
 'DCGS0069',
 'DCGS0004',
 'DCGS0003',
 'D',
 'CRUK',
 'C2',
 'BANK CHARGES',
 'B',
 'AMAZONFEE']

In [ ]:
data['non_sale'] = np.where(data.stock_code.isin(non_sale), 1, 0)

In [ ]:
non_sale_revenue = data[data['non_sale']==1].groupby(['stock_code','description'])['revenue'].sum().reset_index()
non_sale_revenue

In [ ]:
non_sale_revenue.revenue.sum()

All these rows are not relevant to the product range analysis, we can remove them.

In [ ]:
data = data.query('non_sale == 0')

In [ ]:
data = data.drop('non_sale', axis=1)
data.head()

**Investigating the returned items:**

In [ ]:
returned_items = data[data['return'] == 1].groupby(['description']).agg({'return':'count','revenue':'sum'}).reset_index()
returned_items['percentage'] = np.round((returned_items['return']/returned_items['return'].sum())*100, decimals=2)
returned_items.columns = ['description','total_returned','total_revenue','percentage']
returned_items.sort_values('total_returned',ascending=False).head(10).reset_index(drop=True)

These are the products that customers mostly returns.

In [ ]:
data['return'].value_counts(normalize=True).reset_index()

1.65% of the products are returned.

In [ ]:
data[data['return'] == 1]['revenue'].sum()

In [ ]:
returns = data.groupby('return')['revenue'].sum().reset_index()
returns['revenue'] = returns['revenue'].abs()
returns

In [ ]:
fig = px.pie(returns, values='revenue', names='return', title='Sold VS returned products')
fig.show()

The returned products are 2.31% out of the store revenues.  
Let's see the monthly distribution:

In [ ]:
monthly_return = data[data['return'] == 1].groupby(['month'])['description'].count().reset_index()
total = monthly_return['description'].sum()
monthly_return['percentage'] = np.round((monthly_return['description'] / total) * 100, decimals=2)
monthly_return = monthly_return.rename(columns={'description':'total_return'})
monthly_return

In [ ]:
fig = px.bar(monthly_return,x='month',y='total_return', title='Montly returned products',text='total_return')
fig.update_layout(xaxis = dict(tickmode = 'array',tickvals = monthly_return['month']))
fig.show()

In [ ]:
monthly_return.query('month == ["2019-10","2019-11"]')['percentage'].sum()

October and November has the most returns!.  
27% of the return are in these months.

#### Checking for outliers

In [ ]:
border_list=[]
for parameter in ['unit_price','quantity']:
    ninety_five = np.percentile(data[parameter], 98)
    five = np.percentile(data[parameter], 2)
    border_list.append([parameter,five,ninety_five])

border_list

In [ ]:
borders = pd.DataFrame(border_list , columns=['parameter', 'min', 'max'])

In [ ]:
borders

#### Unit price:

In [ ]:
data['unit_price'].describe()

In [ ]:
price_outliers = data.query(' 0.39 > unit_price or unit_price > 12.75')
price_outliers.sort_values('unit_price').head()

In [ ]:
price_outliers.sort_values('unit_price', ascending=False).head()

In [ ]:
print('The unit price outliers are','{:.2%}'.format(len(price_outliers)/len(data)),'of the data')

In [ ]:
print('The unit price outliers revenue is',price_outliers.revenue.sum())

In [ ]:
print('The unit price outlier are' ,'{:.2%}'.format(price_outliers.revenue.sum()/ data.revenue.sum()), 'of the revenues')

In [ ]:
price_outliers = data['unit_price']
price_clean = data.query(' 0.39 < unit_price < 12.75')

In [ ]:
fig = px.histogram(price_outliers, x='unit_price', title='Unir price w/outliers')
fig.show()

In [ ]:
fig = px.histogram(price_clean, x='unit_price', title='Unit price w/o outliers', nbins = 10)
fig.show()

In [ ]:
fig = plt.figure(figsize=(16,8))
x_values = pd.Series(range(0,len(price_outliers)))
plt.scatter(x_values, price_outliers) 
plt.title('Unit price w/outliers', fontdict = {'size':16})
plt.xlabel('count', fontdict={'size':14})
plt.ylabel('unit_price', fontdict={'size':14})
plt.show()

In [ ]:
fig = plt.figure(figsize=(16,8))
x_values = pd.Series(range(0,len(price_clean)))
plt.scatter(x_values, price_clean['unit_price']) 
plt.title('Unit price w/o outliers', fontdict = {'size':16})
plt.xlabel('count', fontdict={'size':14})
plt.ylabel('unit_price', fontdict={'size':14})
plt.show()

#### Quantity:

In [ ]:
data.quantity.describe()

In [ ]:
quantity_outliers = data.query(' 1 > quantity or quantity > 72')
quantity_outliers.sort_values('quantity').head()

In [ ]:
print('The quantity outlier are','{:.2%}'.format(len(quantity_outliers)/len(data)), 'of the data')

In [ ]:
print('The quantity outlier are','{:.2%}'.format(quantity_outliers.revenue.sum()/data.revenue.sum()), 'of the revenue')

In [ ]:
quantity_outliers = data['quantity']
quantity_clean = data.query(' 1 < quantity < 72')

In [ ]:
fig = px.histogram(quantity_outliers, x='quantity', title='Quantity w/outliers')
fig.show()

In [ ]:
fig = px.histogram(quantity_clean, x='quantity', title='Quantity per invoice w/o outliers', nbins=10)
fig.show()

In [ ]:
fig = plt.figure(figsize=(16,8))
x_values = pd.Series(range(0,len(quantity_outliers)))
plt.scatter(x_values, quantity_outliers) 
plt.title('Quantity w/outliers', fontdict = {'size':16})
plt.xlabel('count', fontdict={'size':14})
plt.ylabel('quantity', fontdict={'size':14})
plt.show()

In [ ]:
fig = plt.figure(figsize=(16,8))
x_values = pd.Series(range(0,len(quantity_clean)))
plt.scatter(x_values, quantity_clean['quantity']) 
plt.title('Quantity w/o outliers', fontdict = {'size':16})
plt.xlabel('count', fontdict={'size':14})
plt.ylabel('quantity', fontdict={'size':14})
plt.show()

I choose not to remove the outliers as they have a significant influence on the revenue, between 5-20%.

Let's look at the data after all the changes:

In [ ]:
data.describe(include='all')

**findings:**
- Our data starts on 2018-12-03 and ends on 2019-11-30 - total of a year.
- We had 2 columns with missing values - description (common name from the stock code), customer_id (unregister).
- We have 22,240 unique invoices.
- 3796 unique products.
- 4298 unique customers.
- Most of the users are unregistered.
- Invoice 573585 is top invoice with 1112 rows (It has no customer id related).
- WHITE HANGING HEART T-LIGHT HOLDER (85123A) is the best seller product (with 2284 rows).
- The best sale date is '2019-11-27'.
- Weekday is more common then weekends.

** **

**Statistical findings:**
- Avg quantity is 3.
- Avg price is 2.
- Avg revenue is 9.8.
- The median is lower than the avg and the min/max price and quantity has unusual values - we should check for outliers.

### Perform more exploratory data analysis (EDA) to gain insights into the product range and customer behavior.


In [ ]:
data.columns

In [ ]:
#function for presenting a hist and all data distribution
def distribution_percentile(parameter):
    fig = px.histogram(data, x=parameter, title=parameter)
    fig.show()
    if data[parameter].mean()>data[parameter].median():
        print('The clean data distribution is skewed to the right')
    else:
        print('The clean data distribution is skewed to the left')
    print('---------------------------------------')
    print('Statistics on: {}'.format(parameter))
    print(data[parameter].describe())
    print('')

In [ ]:
for parameter in ['date', 'revenue']:
    distribution_percentile(parameter)

### EDA and proccessing conclusions:

- Most of the product prices are very low, the meadian is 2 dollars.
- 75% of the prices are lower than 4.13 dollars.
- The majority of customers tend to make purchases of smaller amounts.
- Quantity of 1 and 9  products are the most common in the store.
- 75% of the quantity is less than 10 products.
- Most of the store sells are during holidays/winter time.
- November 14-15th has the most orders.
- During the last week of the year and on the 19-25/04 the web wasn't active- maybe for stocktaking.
- November is the top month.
- Wednesday and Friday has the highest amount of orders.
- The store website is closed on Sundays or people prefer to spend their time outside and less buying online.

### Build a correlation matrix and display it.

In [ ]:
corr = data.corr()
plt.figure(figsize=(9, 6))
ax = sns.heatmap(corr, annot=True, square= True)
ax.set_title('Features correlation matrix', fontsize=16)
plt.show()

- Medium positive correlation:
    - Revenue and Quantity (0.52)
- Low positive correlation:
    - Revenue and Unit_price (0.18)  
    If the unit price \ quantity rises the revenue will too.

## Analyze the product range

### Group products by category, similar name, price range, and other relevant factors. 

In [ ]:
data.head()

#### Price range:

Let's investigate the prices without ouliers: 
As we can see from the above hist the unit price is skewed to the left.  
Most of the prices are around 0.5-3.

In [ ]:
price_clean.head()

In [ ]:
fig = px.histogram(price_clean, x='unit_price', title= 'unit price distribution', nbins=10)
fig.show()

In [ ]:
print('Max unit price is:', price_clean.unit_price.max())
print('Min unit price is:', price_clean.unit_price.min())

we can see that our product prices range is from 0.4-12.72.  
224K are between 0.4-2 dollars.  
135.5K are between 2-4 dollars.  
72K are between 4-6 dollars.  
49K are between 6-12.72 dollars.  
Our customers tends to prefer cheap products, between 0.4-2 dollars.

In [ ]:
def price_category(price):
    if  price < 0:
        return 'negative'
    elif 0 < price < 2:
        return 'less than 2 dollars'
    elif 2 <= price < 4:
        return '2-4 dollars'
    elif 4<= price < 6:
        return '4-6 dollars'
    else:
        return '6 dollars or more'

In [ ]:
data['price_range'] = data['unit_price'].apply(price_category)

In [ ]:
data.head()

We divided the product to 4 groups:
- Less than 2 dollars
- 2-4 dollars
- 4-6 dollars
- more than 6 dollars

In [ ]:
price_range_count = data.price_range.value_counts().reset_index()
price_range_count

In [ ]:
price_range_revenue = data.groupby('price_range')['revenue'].sum().reset_index()
price_range_revenue['per'] = price_range_revenue['revenue']/price_range_revenue['revenue'].sum()
price_range_revenue.sort_values('revenue', ascending=False)

In [ ]:
fig = px.pie(price_range_revenue, values='revenue', names='price_range', title='Revenue by price range')
fig.show()

In [ ]:
price_range_quantity = data.groupby('price_range')['quantity'].sum().reset_index()
price_range_quantity['per'] = price_range_quantity['quantity']/price_range_quantity['quantity'].sum()
price_range_quantity.sort_values('quantity', ascending=False)

In [ ]:
fig = px.pie(price_range_quantity, values='quantity', names='price_range', title='Quantity by price range')
fig.show()

Most of the purchased products costs less than 2 dollars (37.5% of the revenues).  
There are also many products with price range of 2-4 dollars (27% of the revenues).  


### Calculate metrics such as revenue, revenue per month, and quantity per month for each product.  

In [ ]:
print('The total revenue is:',data.revenue.sum())

### Returned:

In [ ]:
return_data = data[data['return']==1]

In [ ]:
print('The total returned revenue is:',return_data['revenue'].sum())

In [ ]:
print('The total product returned:', return_data['description'].nunique())

In [ ]:
return_data['description'].nunique()/data['description'].nunique()

In [ ]:
print('The total returned items are:',return_data.quantity.sum())

In [ ]:
return_data.quantity.sum()/data.quantity.sum()

In [ ]:
print('The total amount of returned invoices is:', return_data['invoice_no'].nunique())

In [ ]:
return_data['invoice_no'].nunique()/data['invoice_no'].nunique()

### Sold:

In [ ]:
sales = data[data['return']==0]

In [ ]:
print('The total sales revenue is:','{:.2f}'.format(sales['revenue'].sum()))

In [ ]:
print('The total product sold:', sales['description'].nunique())

In [ ]:
print('The total items that was sold are:',sales['quantity'].sum())

In [ ]:
print('The total amount of invoices is:', sales['invoice_no'].nunique())

In [ ]:
revenue_date = sales.groupby(['date'])['revenue'].sum().reset_index()
revenue_date.sort_values('revenue',ascending = False).head(10)

In [ ]:
revenue_date.median()

In [ ]:
fig = px.line(revenue_date, x="date", y="revenue", title='Revenue over time')
fig.show()

2019-09-18 and 2019-11-12 has the highest revenues.  
As we already know the winter/holiday times are the most profitable period.  

Let's see the monthly revenue:

In [ ]:
revenue_month = data.groupby('month')['revenue'].sum().reset_index()
revenue_month['per'] = (revenue_month['revenue']/revenue_month['revenue'].sum())*100
revenue_month['prc_change'] = revenue_month['revenue'].pct_change()
revenue_month

In [ ]:
fig = px.line(revenue_month, x='month', y='revenue', markers=True)
fig.update_layout(xaxis = dict(tickmode = 'array',tickvals = revenue_month['month']))
fig.show()

In [ ]:
change = (revenue_month.loc[11, 'revenue'] - revenue_month.loc[0, 'revenue']) / revenue_month.loc[0, 'revenue'] * 100
print('The revenue from Janury to November increased in:',np.round(change,2),'%')

In [ ]:
fig = px.bar(revenue_month, x='month', y='revenue', title='Revenue by months')
fig.update_layout(xaxis = dict(tickmode = 'array',tickvals = revenue_month['month']))

fig.show()

In [ ]:
print('Sep-Nov total revenue percentage:' ,
      '{:.2f}'.format(revenue_month[revenue_month.month.isin(['2019-09','2019-10','2019-11'])]['per'].sum()), "%")

In [ ]:
revenue_month_price_range = data.groupby(['month','price_range'])['revenue'].sum().reset_index()
revenue_month_price_range['per'] = np.round((revenue_month_price_range['revenue']/revenue_month_price_range['revenue'].sum())*100,2)
revenue_month_price_range.sort_values('revenue', ascending = False).head(10)

In [ ]:
fig = px.bar(revenue_month_price_range, x='month', y='revenue',color='price_range', title='Revenue by months',barmode='group')
fig.show()

November is the most profitable month!.  
The most profitable price range are 0.4-4 dollar in November.  
Less than 2 dollars are the most profitable products and 2-4 dollars is a bit behind.

Let's check the day of the week:

In [ ]:
day_of_week =  data.groupby('day_of_week').agg({'revenue':['sum','mean']}).reset_index()
day_of_week.columns = ['day_of_week', 'total_revenue', 'average_revenue']
day_of_week['per'] = (day_of_week['total_revenue']/day_of_week['total_revenue'].sum())*100
day_of_week['avg_per'] = (day_of_week['average_revenue']/day_of_week['average_revenue'].sum())*100
day_of_week

In [ ]:
data.day_of_week.describe()

In [ ]:
data.day_of_week.unique()

In [ ]:
fig = px.bar(day_of_week, x='day_of_week', y='total_revenue', title='Total revenue for each day of the week')
fig.update_xaxes(
    tickvals=[0, 1, 2, 3, 4, 5], 
    ticktext=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday'])
fig.show()

In [ ]:
fig = px.bar(day_of_week, x='day_of_week', y='average_revenue',title='Avg revenue for each day of the week', color_discrete_sequence=['green'])
fig.update_xaxes(
    tickvals=[0, 1, 2, 3, 4, 5], 
    ticktext=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday'])
fig.show()

The total and avg revenue on Friday and Wednesday are the highest.  
The total and avg revenue on Monday is the lowest.  
We notice that there aren't any sales on Sundays.  

Will will check the weekend vs weekdays later in the hypothesis.

In [ ]:
quantity_fig = data.groupby('month')['quantity'].sum().reset_index()
revenue_fig = data.groupby('month')['revenue'].sum().reset_index()

In [ ]:
trace1 = go.Scatter(
    x=revenue_fig['month'],
    y=revenue_fig['revenue'],
    name='Revenue'
)

trace2 = go.Scatter(
    x=quantity_fig['month'],
    y=quantity_fig['quantity'],
    name='Quantity',
    yaxis='y2',
)

fig = go.Figure()
fig.add_trace(trace1)
fig.add_trace(trace2)

fig.update_layout(
    title='Revenue VS quantity over time',
    xaxis= dict(title='month'),
    yaxis=dict(
        title='Revenue'
    ),
    yaxis2=dict(
        title='Quantity',
        overlaying='y',  
        side='right'
    )
)

fig.show()


In [ ]:
unit_price_fig = data.groupby('month')['unit_price'].median().reset_index()

In [ ]:
trace1 = go.Scatter(
    x=revenue_fig['month'],
    y=revenue_fig['revenue'],
    name='Revenue'
)

trace2 = go.Scatter(
    x=unit_price_fig['month'],
    y=unit_price_fig['unit_price'],
    name='Unit price',
    yaxis='y2'  
)

fig = go.Figure()
fig.add_trace(trace1)
fig.add_trace(trace2)

fig.update_layout(
    title='Revenue VS unit_price over time',
    xaxis= dict(title='month'),
    yaxis=dict(
        title='Revenue'
    ),
    yaxis2=dict(
        title='Avg unit price',
        overlaying='y',  
        side='right'
    )
)

fig.show()


We can see that the revenue increased by the quantity sold and not the avg unit_price.

### Identify the most and least popular products, as well as those with the highest and lowest revenue.  

**The top products:**

In [ ]:
data[['stock_code','description']].describe()

**Top 10 products:**

In [ ]:
top10 = data.groupby(['stock_code','description']).agg({'invoice_no': 'nunique', 'unit_price': 'median','quantity':'sum','revenue':'sum'}).reset_index()
top10.rename(columns={'invoice_no': 'total_orders', 'unit_price':'avg_price','quantity': 'total_quantity', 'revenue':'total_revenue'}, inplace=True)

**Top 10 products ordered:**

In [ ]:
top_orders = top10.sort_values('total_orders', ascending=False).head(10)
top_orders

In [ ]:
top_orders['total_orders'].sum()

In [ ]:
print('The top 10 percentage from total orders is :',
      '{:.2%}'.format(top_orders['total_orders'].sum()/top10['total_orders'].sum()))

**Top 10 products by revenue:**

In [ ]:
top_revenue = top10.sort_values('total_revenue',ascending = False).head(10)
top_revenue

In [ ]:
print('Top 10 total revenue is:',top_revenue['total_revenue'].sum())

In [ ]:
print('The top 10 percentage from total revenue is :',
      '{:.2%}'.format(top_revenue['total_revenue'].sum()/top10['total_revenue'].sum()))

**Top 10 products by quantity:**

In [ ]:
top10.sort_values('total_quantity',ascending = False).head(10)

The store best product: 'WHITE HANGING HEART T-LIGHT HOLDER’.  
Total ordered 2210, quantity: 33,871 pcs, revenue: 95,544.55  

Top revenue: REGENCY CAKESTAND 3 TIER with 152,902.74.  
Top quantity: POPCORN HOLDER with 51,477.  

Our most popular products related to parties, design and decoration.  

**The 10 least products:**

In [ ]:
least10 = data.query('quantity > 0').groupby(['stock_code','description']).agg({'invoice_no': 'nunique', 'unit_price': 'median','quantity':'sum', 'revenue':'sum'}).reset_index()
least10.rename(columns={'invoice_no': 'total_orders', 'unit_price':'avg_price', 'quantity':'total_quantity','revenue':'total_revenue'}, inplace=True)

**Least 10 products ordered:**

In [ ]:
least_revenue = least10.sort_values(['total_orders','total_revenue']).head(10)
least_revenue

In [ ]:
least_revenue['total_revenue'].sum()

In [ ]:
print('The top 10 percentage from total revenue is :',
      '{:.2%}'.format(least_revenue['total_revenue'].sum()/least10['total_revenue'].sum()))

**Least 10 products by quantity:**

In [ ]:
least10.sort_values(['total_quantity','avg_price']).head(10)

In [ ]:
least10[least10['total_orders'] == 1]['description'].nunique()

In [ ]:
print('{:.2%}'.format(least10[least10['total_orders'] == 1]['description'].nunique()/data.description.nunique()),'of the products ordered just once.')

HEN HOUSE W CHICK IN NEST is the least profitable product.  
Total ordered 1, quantity: 1, revenue: 0.42 dollars.  

Customers buys less cards.

### RFM analysis

**Recency:** How recently has the customer made a transaction with us.  
**Frequency:** How frequent is the customer in ordering/buying some product from us.  
**Monetary:** How much does the customer spend on purchasing products from us.  

In [ ]:
customer_data = data.query('customer_id != "unregister"')
customer_data.head()

In [ ]:
# calculating recency
df_recency = customer_data.groupby(by='customer_id', as_index=False)['date'].max()
df_recency.columns = ['CustomerName', 'LastPurchaseDate']
recent_date = df_recency['LastPurchaseDate'].max()
df_recency['Recency'] = df_recency['LastPurchaseDate'].apply(lambda x: (recent_date - x).days)

df_recency.head()

In [ ]:
# calculating frequency
frequency_df = customer_data.groupby(by=['customer_id'], as_index=False)['date'].count()
frequency_df.columns = ['CustomerName', 'Frequency']

frequency_df.head()

In [ ]:
# Calculating monetary
customer_data['Total'] = customer_data['unit_price']*customer_data['quantity']
monetary_df = customer_data.groupby(by='customer_id', as_index=False)['Total'].sum()
monetary_df.columns = ['CustomerName', 'Monetary']

monetary_df.head()

In [ ]:
# merging all
rf_df = df_recency.merge(frequency_df, on='CustomerName')
rfm_df = rf_df.merge(monetary_df, on='CustomerName').drop(columns='LastPurchaseDate')

rfm_df.head()

In [ ]:
rfm_df['R_rank'] = rfm_df['Recency'].rank(ascending=False)
rfm_df['F_rank'] = rfm_df['Frequency'].rank(ascending=True)
rfm_df['M_rank'] = rfm_df['Monetary'].rank(ascending=True)

# normalizing the rank of the customers
rfm_df['R_rank_norm'] = (rfm_df['R_rank']/rfm_df['R_rank'].max())*100
rfm_df['F_rank_norm'] = (rfm_df['F_rank']/rfm_df['F_rank'].max())*100
rfm_df['M_rank_norm'] = (rfm_df['F_rank']/rfm_df['M_rank'].max())*100

rfm_df.drop(columns=['R_rank', 'F_rank', 'M_rank'], inplace=True)

rfm_df.head()

Formula used for calculating rfm score is : 0.15 X Recency score + 0.28 X Frequency score + 0.57 X Monetary score

In [ ]:
# Calculating RFM score

rfm_df['RFM_Score'] = (0.15*rfm_df['R_rank_norm'])+(0.28*rfm_df['F_rank_norm'])+(0.57*rfm_df['M_rank_norm'])
rfm_df['RFM_Score'] *= 0.05
rfm_df = rfm_df.round(2)

rfm_df[['CustomerName', 'RFM_Score']].head(7)

Rating Customer based upon the RFM score:  
RFM score > 4.5 : **Top customer**  
4.5 > RFM score > 4 : **High Value customer**  
4> RFM score >3 : **Medium value customer**  
3> RFM score >1.6 : **Low-value customer**  
RFM score <1.6 :**Lost customer**  

In [ ]:
rfm_df["Customer_segment"] = np.where(rfm_df['RFM_Score'] > 4.5, "Top Customers",
                            (np.where(rfm_df['RFM_Score'] > 4, "High value Customer",
                            (np.where(rfm_df['RFM_Score'] > 3, "Medium Value Customer",
                            np.where(rfm_df['RFM_Score'] > 1.6,'Low Value Customers', 'Lost Customers'))))))
rfm_df[['CustomerName', 'RFM_Score', 'Customer_segment']].head(20)


Visualizing the customer segments:

In [ ]:
fig = go.Figure(data=[go.Pie(labels=rfm_df['Customer_segment'], hole=.3)])
fig.update_layout(title_text='Customer segments')
fig.show()

In [ ]:
rfm_df['CustomerName'] = rfm_df['CustomerName'].astype('object')

In [ ]:
rfm_revenue = rfm_df[['CustomerName','Customer_segment']].merge(data[['revenue','customer_id']], left_on='CustomerName', right_on='customer_id', how='left')
rfm_revenue = rfm_revenue.drop('customer_id',axis=1)
rfm_revenue.head()

In [ ]:
total_rfm_rev = rfm_revenue.groupby('Customer_segment')['revenue'].sum().reset_index()
total_rfm_rev['per_revenue'] = np.round(total_rfm_rev['revenue']/rfm_revenue['revenue'].sum()*100,2)
total_rfm_rev.sort_values('revenue',ascending=False)

In [ ]:
fig = go.Figure(data=[go.Pie(labels=total_rfm_rev['Customer_segment'], 
                             values=total_rfm_rev['revenue'], 
                             hole=.3,
                             marker=dict(colors=['#ab62fa', '#626ff9', '#ee563b', '#00cc96', '#ffa05a']))])

fig.update_layout(title_text='Customer segments by revenue')
fig.show()


Checking how much users participate in the RFM analysis:

In [ ]:
unreg_inv = data[(data.customer_id =='unregister') & (data['return'] ==0)]

In [ ]:
print('Total unregistered invoices:',unreg_inv['invoice_no'].nunique())

In [ ]:
print('Total unregistered revenue','{:.2f}'.format(unreg_inv['revenue'].sum()))

In [ ]:
print('Unregisters are','{:.2%}'.format(unreg_inv['revenue'].sum()/data[data['return'] ==0]['revenue'].sum()),'of the total revenue')

In [ ]:
(data[(data.customer_id =='unregister') & (data['return'] ==0)]['invoice_no'].nunique()/data.invoice_no.nunique())*100

Most of our customers (registered) are lost or low value customers.  
We should consider change our registration police in order to increase retention rate.

## Machine Learning:

In [ ]:
data.columns

### Use classification models to identify the most and least profitable products and optimize the product range.


In oreder to get better results let's take out the outliers:

In [ ]:
borders

In [ ]:
data_clean = data.query('1<= quantity<= 72 &  0.39 <= unit_price <= 12.75')
data_clean.head()

In [ ]:
# create a new DataFrame with the relevant columns
product_data = data_clean.groupby('stock_code').agg({'unit_price':'mean', 'quantity':'sum'}).reset_index()

# normalize the data
product_data['unit_price'] = (product_data['unit_price'] - product_data['unit_price'].mean()) / product_data['unit_price'].std()
product_data['quantity'] = (product_data['quantity'] - product_data['quantity'].mean()) / product_data['quantity'].std()


In [ ]:
product_data.head()

In [ ]:
xc = product_data.drop('stock_code', axis=1)

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

linked= linkage(xc, method='ward')

plt.figure(figsize=(15,10))
dendrogram(linked, orientation= 'top')
plt.title('Hierarchical clustering for the store')
plt.show()

In [ ]:
# perform k-means clustering
kmeans = KMeans(n_clusters=3, random_state=0).fit(product_data[['unit_price', 'quantity']])

# add the cluster labels to the DataFrame
product_data['cluster'] = kmeans.labels_

# plot the results
fig = plt.figure(figsize=(14,6))
plt.scatter(product_data['unit_price'], product_data['quantity'], c=product_data['cluster'])
plt.xlabel('Average Unit Price')
plt.ylabel('Total Quantity Sold')
plt.title('K-means clusters')
plt.show()

According to the hierarchical clustering we should divide the data to 3 groups.  

Let's check the groupss distribution:

In [ ]:
for i in ['unit_price','quantity']:
    fig = px.histogram(product_data, x=i, color='cluster',title=i, barmode='overlay')
    fig.show()

After normalizing the data, we divided the data to 3 clusters using K-means.  
We received 3 different groups:
1. Low quantity and low unit price
2. Low quantity and higher unit price
3. Higher quantity and low unit price

We can learn from the histograms that:
- The clusters aren't equal, cluster 0 is bigger than 1,2.
- Cluster 0,1 customer tends to buy small amount of products, while cluster 2 buys more.
- Cluster 1 buy more expensive products but with low quantity.
- Cluster 2 buy more products but in lower price.
- Cluster 0 buy low price and low quantity products.

In [ ]:
data_cluster = data_clean.merge(product_data[['stock_code','cluster']], on='stock_code', how='left')

In [ ]:
data_cluster.groupby('cluster')['description'].nunique()

In [ ]:
data_cluster[data_cluster['cluster']==2]['description'].unique()

## Test the statistical hypotheses:

### Hypothesis: There is a significant difference in sales between weekdays and weekends.  
Test: Two-sample t-test comparing the mean sales on weekdays versus weekends.

    H0: The weekdays and weekend mean revenue is the same.
    H1: The weekdays and weekend mean revenue is different.

In [ ]:
weekday_sales = data[data['weekday_or_weekend'] == 'Weekday']['revenue']
weekend_sales = data[data['weekday_or_weekend'] == 'Weekend']['revenue']

In [ ]:
def testing_average(sample1,sample2,alpha=0.05):
    #checking normality
    sample1=weekday_sales
    sample2=weekend_sales
    stat1, p1_norm=st.shapiro(sample1)
    stat2, p2_norm=st.shapiro(sample2)
    if p1_norm > alpha and p2_norm > alpha:
        print('p1:',p1_norm,'p2:', p2_norm)
        #then both normal, perform t-test.
        #for t-test also should check the equality of variances
        statslev,p_levene=st.levene(sample1, sample2)
        print('p_levene',p_levene)
        if p_levene < alpha:
        #variances are not equal
            statist,p_value=st.ttest_ind(sample1, sample2,equal_var=False)
        else:
            statist,p_value=st.ttest_ind(sample1, sample2,equal_var=True) 
    else:
        statist,p_value=st.mannwhitneyu(sample1, sample2)
    print('p_value:', p_value)
    if p_value<alpha:
        print('Rejecting the null hypothesis for weekdays and weekends')
    else:
        print('Fail to Reject the null hypothesis for weekdays and weekends')

In [ ]:
testing_average(weekday_sales,weekend_sales)

The p-value is lower than 0.05.  
We reject the null hypothesis, the 2 groups are different.  
It means there is a different in the revenue between the weekdays and to weekends.

In [ ]:
weekend_weekday_inv = data.groupby('weekday_or_weekend')['invoice_no'].nunique().reset_index()
weekend_weekday_inv

In [ ]:
fig = px.bar(weekend_weekday_inv, x='weekday_or_weekend', y='invoice_no', title='Total invoices per weekend/day')
fig.show()

In [ ]:
weekend_weekday_rev = data.groupby('weekday_or_weekend').agg({'revenue':['sum','mean']}).reset_index()
weekend_weekday_rev.columns = ['weekday_or_weekend','total_revenue','avg_revenue']
weekend_weekday_rev['per'] = (weekend_weekday_rev['total_revenue']/weekend_weekday_rev['total_revenue'].sum())*100
weekend_weekday_rev['per_avg']=(weekend_weekday_rev['avg_revenue']/weekend_weekday_rev['avg_revenue'].sum())*100
weekend_weekday_rev

In [ ]:
fig = px.bar(weekend_weekday_rev, x='weekday_or_weekend', y='total_revenue', title='Total revenue per weekend/day')
fig.show()

In [ ]:
fig = px.bar(weekend_weekday_rev, x='weekday_or_weekend', y='avg_revenue', title='Avg revenue per weekend/day', color_discrete_sequence=['green'])
fig.show()

As we can see the total invoices and revenue is Significantly higher during the weekdays.  
But the avg revenue in the weekends is a bit higher than the weekdays.

### Hypothesis: There is a correlation between the quantity of items purchased and the total sales.  
Test: Pearson's correlation coefficient test between the Quantity and Unit Price columns.

In [ ]:
correlation_coefficient, p_value = pearsonr(data['quantity'], data['revenue'])

print('Correlation coefficient:', correlation_coefficient)
print('p-value:', p_value)

The corr coefficient is 0.54, we can say there is a medium relation between the quantity and revenue.  
If the quantity rises the revenue should too.  
The p-value is less than 0.05 then the correlation is considered statistically significant.

### Hypothesis: There is a significant difference in sales between different product categories.  
Test: Tukey's Honestly Significant Difference (HSD) test make multiple comparisons to determine which specific groups are different.

Let's test the hypothesis if there is a statistically significant difference between the different groups:

    H0: The revenue between the price_range groups are the same.
    H1: The revenue between the price_range groups are different.

In [ ]:
tukey = pairwise_tukeyhsd(endog=data['revenue'], groups=data['price_range'], alpha=0.05)

print(tukey)

We recived the difference in means between each pair of groups, the confidence interval of this difference, and whether or not the difference is significant.  

In all the pair we rejected the null hypothesis.  
The revenue between the groups is different.



After making 6 tests on the same data we might have an error result so we should use Bonferroni correction.  
We will divide alpha by 6: To check if we have any error (type 1 or type 2 errors).

In [ ]:
tukey = pairwise_tukeyhsd(endog=data['revenue'], groups=data['price_range'], alpha=(0.05/6))

print(tukey)

We can't find any error or difference.  
We still reject all the null hypothesis.

## Conclusion:

**Fixing the data:**  
- We rename the columns as snake_case
- Changed the columns type
- Added month, day_of_week, weekday_weekend, revenue, price_range columns
- Fix inconsistent values between stock_code and product name - with the most common name.
- Check outliers according to price and quantity (2%).
- Convert all the stock_code to upper case.
- Add filter column (return, damaged, non-sale) for further investigation.

**findings:**  
- Our data starts on 2018-12-03 and ends on 2019-11-30 - total of a year.
- We had 2 columns with missing values - description (common name from the stock code), customer_id (unregister).
- We have 22,240 unique invoices.
- 3796 unique products.
- 4298 unique customers.
- Most of the users are unregistered.
- Invoice 573585 is top invoice with 1112 rows (It has no customer id related).
- WHITE HANGING HEART T-LIGHT HOLDER (85123A) is the best seller product (with 2284 rows).
- The best sale date is '2019-11-27'.
- Weekday is more common then weekends.

**Statistical findings:**  
- Avg quantity: 3.
- Avg price: 2.
- Best month: November.
- Best day: Friday and Wednesday.
- Worst day: Monday.
- Avg daily revenue 28,027.
- Total itema sold: 5,187,066.
- Total revenue: 9,353,024
- The median is lower than the avg and the min/max price and quantity has unusual values.

**EDA findings:**  
- Most of the product prices are very low, the avg is 2 dollars.
- 75% of the prices are lower than 4.13 dollars.
- The majority of customers tend to make purchases of smaller amounts.
- Quantity of 1 and 9  products are the most common in the store.
- 75% of the quantity is less than 10 products.
- Most of the store sells are during holidays/winter time.
- November 14-15th has the most orders.
- During the last week of the year and on the 19-25/04 the web wasn't active- maybe for stocktaking
- November is the top month.
- Wednesday and Friday has the highest amount of orders.
- The store website is closed on Sundays or people prefer to spend their time outside and less buying online.
- The revenue increased in 123% from December 2018 to November 2019.
- 2019-09-18 and 2019-11-12 has the highest revenues.
- The revenue increased by the quantity sold and not by avg unit_price.


**Correlation findings:**  
- Medium positive correlation:
    - Revenue and Quantity (0.52)
- Low positive correlation:
    - Revenue and Unit_price (0.18)  
    If the unit price / quantity rises the revenue will too.

**Revenue findings:**  
- By time: winter/ holidays period are the most profitable time of the year - between September to November.
- By month: November is the most profitable month of the year.  
            September and October has higher revenue compered to other months.
- By day: Friday and Wednesday has the highest total and avg revenues.  
          Monday has the lowest total and avg revenues.  
          There aren't any sales on Sundays.  
- The revenue increased by the quantity sold (123% from December 2018 to November 2019).  

**Price range findings:** 
We divided the product to 4 groups:
1. Less than 2 dollar
2. 2-4 dollars
3. 4-6 dollars
4. more than 6 dollars

Most of the purchased products 

- Most of the company products costs less than 2 dollars (37.5% of the revenues).
- There are also many products with price range of 2-4 dollars (27% of the revenues).
- During all months the products that cost less than 2 dollars are the most profitable one.
- We can see that in quantity matters customers buys more cheap products.  

**Best and worst sellers:**  
Best seller product: 'WHITE HANGING HEART T-LIGHT HOLDER’.  
Total ordered 2210, quantity: 33,871 pcs, revenue: 95,544.55.

Top revenue: REGENCY CAKESTAND 3 TIER with 152,902.74.  
Top quantity: POPCORN HOLDER with 51,477.

Our most popular products related to parties, design and decoration.  

Worst sellers product: ‘HEN HOUSE W CHICK IN NEST’.  
Total ordered 1, quantity: 1, revenue: 0.42 dollars.  
Less often purchased: cards.  

**RFM findings:**  
Most of our customers (registered) are lost (31%) or low value customers (30%).
We should consider change our registration police in order to increase retention rate.

**Machine Learning findings:**  
After normalizing the data, we divided the data to 3 clusters using K-means.  
We received 3 different groups:
1. Low quantity and low unit price
2. Low quantity and higher unit price
3. Higher quantity and low unit price

We can learn from the histograms that:  
- The clusters aren't equal, cluster 0 is bigger than 1,2.
- Cluster 0,1 customer tends to buy small amount of products, while cluster 2 buys more.
- Cluster 1 buy more expensive products but with low quantity.
- Cluster 2 buy more products but in lower price.
- Cluster 0 buy low price and low quantity products.

**Statistical hypotheses findings:**  
1. Hypothesis: There is a significant difference in sales between weekdays and weekends.  
using mannwhitneyu test.  
There is a different in the avg revenue between the weekdays and to weekends.  

1. Hypothesis: There is a correlation between the quantity of items purchased and the total sales.  
using Pearson's correlation coefficient test.  
The correlation coefficient is 0.54 - There is a medium relation between the quantity and revenue.  

3. Hypothesis: There is a significant difference in sales between different product categories.  
Using Tukey's Honestly Significant Difference (HSD) test.  
The revenue between all the price ranges is different.  

## Recommendations:

1. **Customer Acquisition and Retention:** As there are users that can purchase without signing up to the website, there could be potential customer retention and loyalty programs targeted towards signed-up customers. Offering exclusive deals or membership rewards can incentivize more customers to sign up and increase customer retention.

2. **Price and Product Range Optimization:** The findings show that most products are very low priced with a significant volume of sales. This might indicate that customers are price-sensitive. We should consider introducing a wider range of products with different price points, especially in the mid to high-price range. This can be done gradually while measuring customer response and revenue impact.

3. **Seasonal and Weekly Promotions:** It seems that there are specific periods, like November, Wednesday and Fridays, where sales are particularly high. We should consider creating seasonal promotions or weekly deals to further boost sales during these periods.

4. **Product Line Expansion:** The top-selling and most profitable products related to parties, designs, and decorations. It may be beneficial to expand this product line and offer more variety within these popular categories.

5. **Improve Product Availability:** The store website seems to be inactive during the last week of the year and on the 19-25/04. We should try to minimize downtime and ensure product availability year-round.

6. **Leverage Machine Learning for Business Strategy:** The identified clusters from your K-means algorithm can help shape the business strategy. For example, for customers who buy small quantities of expensive items (Cluster 1), consider offering product bundles or discounts for multiple purchases to encourage buying in larger quantities. For customers who buy large quantities of cheaper items (Cluster 2), cross-sell or upsell strategies might be effective, promoting slightly higher-priced items that complement their purchases.

7. **Testing and Optimization:** We should keep testing the hypotheses and optimizing based on the results. For example, the finding that there is a difference in sales between weekdays and weekends could be used to optimize the timing of promotions or marketing efforts. Similarly, the correlation between the quantity of items purchased and total sales could be used to develop strategies aimed at increasing the average number of items per purchase.

### References

RFM: https://www.geeksforgeeks.org/rfm-analysis-analysis-using-python/  
Tukey's HSD test: https://www.statology.org/tukey-test-python/  
plotly express for additional graphs adjustments: https://plotly.com/python/pie-charts/  
sStack overflow and chat GPT for general code solving.

### [Link](https://drive.google.com/file/d/1bBhIS8LA9H1jVhlut7SvzF4OTk5DpO5n/view?usp=sharing) to the presenation